In [20]:
# See https://pypsa.readthedocs.io/en/latest/user-guide/contingency-analysis.html#branch-outage-distribution-factors-bodf
import polars as pl
from benchmark_utils import mock_snakemake

DF_CUTOFF = 1e-5


if "snakemake" not in globals():  # noqa: F821
    snakemake = mock_snakemake("compute_bodf")


lines = pl.read_parquet(
    snakemake.input.lines, columns=["line_id", "from_bus", "to_bus", "is_leaf"]
)
ptdf = pl.read_parquet(snakemake.input.ptdf)
ptdf

injection,line_id,factor
i64,i64,f64
1,1,1.0
1,2,-0.558747
1,3,0.441253
1,5,-0.441253
1,6,0.441253
…,…,…
8867,10815,-0.000006
8867,10818,-5.5862e-9
8867,10819,1.9651e-8


In [21]:
# Step 1: Remove leaf lines because we can't have be a contingency
lines = lines.filter(~pl.col("is_leaf")).drop("is_leaf")
lines

line_id,from_bus,to_bus
i64,i64,i64
6457,5559,5563
3265,3962,3963
10688,8617,8618
4800,4802,6574
499,7771,8834
…,…,…
6865,5777,5811
3473,565,4059
60,2363,2448


In [22]:
# Step 2: Compute the difference in PTDF for every from and to bus.
bptdf = (
    lines.rename({"line_id": "outage_line_id"})
    .join(
        ptdf.rename({"injection": "from_bus", "factor": "injection"}),
        on="from_bus",
    )
    .join(
        ptdf.rename({"injection": "to_bus", "factor": "withdrawal"}),
        on=["to_bus", "line_id"],
    )
    .select(
        "outage_line_id",
        "line_id",
        factor=pl.col("injection") - pl.col("withdrawal"),
    )
)
bptdf

outage_line_id,line_id,factor
i64,i64,f64
5384,2,1.5966e-7
5384,3,1.5966e-7
5384,5,-1.5966e-7
5384,6,1.5966e-7
5384,8,-0.000016
…,…,…
2794,10820,-9.5599e-9
1227,10820,8.5695e-8
2796,10821,5.8184e-8


In [23]:
# Step 3: Separate diagonal and off-diagonal entries
diagonal_entries = bptdf.filter(pl.col("outage_line_id") == pl.col("line_id")).drop(
    "line_id"
)
offdiagonal_entries = bptdf.filter(pl.col("outage_line_id") != pl.col("line_id"))
diagonal_entries

outage_line_id,factor
i64,f64
5384,0.563213
6154,0.528895
7688,0.609368
1224,0.930868
1204,0.945297
…,…
5267,0.521666
10821,0.650949
1227,0.79734


In [24]:
# Step 4: Confirm that there are no bridge lines (lines that would cause the grid to split into two unconnected networks)
bridge_lines = diagonal_entries.filter(pl.col("factor") >= (1 - DF_CUTOFF))
if bridge_lines.height > 0:
    raise ValueError(
        f"Grid is not well connected\n{diagonal_entries.filter(pl.col('factor') >= (1 - DF_CUTOFF))}"
    )

In [25]:
# Step 5: Normalize the factors to account for the flow on the line in outage itself
bodf = (
    offdiagonal_entries.join(
        diagonal_entries.select("outage_line_id", denominator=1 - pl.col("factor")),
        on="outage_line_id",
    )
    .with_columns(pl.col("factor") / pl.col("denominator"))
    .drop("denominator")
    .rename({"line_id": "affected_line_id"})
)
bodf

outage_line_id,affected_line_id,factor
i64,i64,f64
5384,2,3.6552e-7
5384,3,3.6552e-7
5384,5,-3.6552e-7
5384,6,3.6552e-7
5384,8,-0.000036
…,…,…
2794,10820,-3.1398e-7
1227,10820,4.2285e-7
2796,10821,0.000005


In [28]:
# Filter out near zeros
bodf = bodf.filter(pl.col("factor").abs() > DF_CUTOFF)
bodf = bodf.sort("outage_line_id", "affected_line_id")
bodf

outage_line_id,affected_line_id,factor
i64,i64,f64
2,3,-1.0
2,5,1.0
2,6,-1.0
2,615,-0.000011
2,616,0.000017
…,…,…
10821,10805,0.002804
10821,10812,-0.000023
10821,10815,-0.000231


In [27]:
bodf.write_parquet(snakemake.output[0])